# 1 Last week we used the activation function. Why is this not used with backpropagation?

__A:__ The function is not differentiable.

# 2 What is the minimum number of hidden neuron layers needed in order to approximate an arbitrary continuous function, and why?

__A:__ One hidden layer is the minimum. "Universal approximation theorem: Any continuous function
can be approximated by a neural network with a single
hidden layer", slide 54 https://www.uio.no/studier/emner/matnat/ifi/INF3490/h18/timeplan/slides/lecture6-1pp.pdf


Without a hidden layer, it is impossible to approximate non-linear separable problems. The first layer gives lines, and the second layer opens up the dimension to two, so e.g. traingles can be made. Furthermore, a compbination of enough traingles can approximate any shape. See slide 12 https://www.uio.no/studier/emner/matnat/ifi/INF3490/h18/timeplan/slides/lecture6-1pp.pdf

<mark> UNCLEAR <br>
    
    
    
# 3 Why do we use a validation set? Describe how the three different cross-validation methods presented in the lecture slides work, and what their advantages and disadvantages are.
We want the model to generalize well, meaning that it works well on other data than the data the model was estimated on. When fitting the model to the full dataset, the model will typically not fit well for other data. In oredr to create a model that generalizes well, the data is split into a training and a validation set. The model is fit to the training set, and is then evaluated on the validation set. The model performance on the validation set describes how well the model generalizes. <br>

Three crossvalidation methods. <br> 
1) Split into training and validation (and possible test). Works as the previous paragraph. <br>
2) K-fold cross-validation. Split into K number of folds. Each fold performs as validation set once, while at the same time the model is fit to the remaining folds. <br>
3) Bootstrapping. Draw random observations with replacement and use the rest as validation. Repeat.

# 4 Implement the MLP shown below, and train it to correctly perform the XOR function.

In [101]:
import numpy as np
import matplotlib.pyplot as plt
class NN:
    """ 2 layer (1 hidden). Single input combination of 2 intergers. Output 2 integers"""
    
    def __init__(self, nodeNumbers, activationFunction, learningRate, targetMatrix, inputMatrix, errorTolerance, \
                 maxIterations, test=False, singleOutput=False):
        self.nodeNumbers, self.activationFunction, self.learningRate, self.targetMatrix, self.inputMatrix, self.errorTolerance,  \
        self.maxIterations, self.test, self.singleOutput= \
        nodeNumbers, activationFunction, learningRate, targetMatrix, inputMatrix, errorTolerance,\
        maxIterations, test, singleOutput
        
        
        self.weightMatrices = []
        
        # If single input
        if self.inputMatrix.ndim == 1:
            self.inputs = self.inputMatrix.tolist()
            self.targets = self.targetMatrix.tolist()
            self.targetLength = self.targets
        else:
            self.numberOfInputs = np.shape(self.inputMatrix)[1]
        
        if not test:
            #print('nodeNumbers',nodeNumbers)
            for nodeNumber in range(len(nodeNumbers[1:])):
                #print('nodeNumber', nodeNumber)
                #print('nodeNumbers[nodeNumber]', nodeNumbers[nodeNumber])
                self.weightMatrices.append(np.random.random_sample((nodeNumbers[nodeNumber+1], \
                                                                    nodeNumbers[nodeNumber]+1)) - .5)          
        else:
            self.weightMatrices.append(np.array(((1., -1., 0.), (1., 0., 1.))))
            self.weightMatrices.append(np.array(((1., 1., 0.), (1., -1., 1.)))) 
            self.inputs = np.array((1, 0, 1))
            
        print('self.weightMatrices',self.weightMatrices) 
            
    def run(self):
        self.forward()
        self.calculateErrors()
        self.iterations = 1
        self.outputForPlot1 = []
        self.outputForPlot2 = []
        while np.abs(self.error2) > self.errorTolerance and self.iterations < self.maxIterations: 
            self.backward()
            self.forward()
            #print('Output: ', self.outputs)
            self.calculateErrors()
            self.iterations += 1
            self.outputForPlot1.append(self.outputs[-1][0])
            self.outputForPlot2.append(self.outputs[-1][1])
        #self.plot()

        print('Iterations: ', self.iterations, '|Error|: ', self.error2, 'Output: ', self.outputs[1])
        
    def runMultipleInput(self):
        self.iterationNumber = 0
        numberOfAdjustmentsDuringIteration = 1
        
        self.errorForPlot = []
        while (self.iterationNumber < self.maxIterations and numberOfAdjustmentsDuringIteration != 0):
            self.iterationNumber += 1
            numberOfAdjustmentsDuringIteration = 0

            #print('targetMatrix', self.targetMatrix)

            for inputNumber in range(self.numberOfInputs):
                #print('inputNumber', inputNumber)
                self.inputs = self.inputMatrix[:,inputNumber]
                print('self.inputs',self.inputs)
                self.targets = self.targetMatrix[inputNumber]
                print('self.targets',self.targets)
                self.targetLength = np.asarray([0])
                # FIX to allow for multidim output
                #print('self.targetMatrix[inputNumber] ', self.targetMatrix[inputNumber] )
                self.forward()
                self.calculateErrors()
                print('self.error2', self.error2)

                #print('self.error2', self.error2)
                
                if abs(self.error2[-1]) > self.errorTolerance:
                    numberOfAdjustmentsDuringIteration +=1
                    self.backward()
                    print('self.weightMatrices', self.weightMatrices)
            self.errorForPlot.append(numberOfAdjustmentsDuringIteration)

        print('Iterations: ', self.iterationNumber, '|Error|: ', self.error2[-1])
        self.plotError()
        
    def plotError(self):
        plt.plot()
        plt.plot(np.arange(len(self.errorForPlot)), self.errorForPlot)
        
    def plot(self):
        fig, (ax, ax2) = plt.subplots(1,2)
        ax.plot(np.arange(len(self.outputForPlot1)), self.outputForPlot1)#, label='Input 1')
        ax2.plot(np.arange(len(self.outputForPlot2)), self.outputForPlot2)#, label='Input 2')
        ax.set_title('Input 1')
        ax2.set_title('Input 22')
        plt.savefig('simple.pdf')
        
    def differentiateActivationFunction(self, weightedInputs):
        if isinstance(weightedInputs, np.ndarray):
            if weightedInputs.ndim == 1:
                onesArray = np.ones(len(weightedInputs))
        else:
            onesArray = 1
        out = self.activationFunction(weightedInputs)*(onesArray - self.activationFunction(weightedInputs))
        #print(self.activationFunction(weightedInputs), onesArray, out)
        #print('out ', out)
        #return 1
        return out
        
    
    def forward(self):
        weightedInputs = []
        #print('self.inputs', self.inputs)
        weightedInputs.append(self.weightMatrices[0] @ self.inputs)
        self.outputs = []
        self.outputs.append(self.activationFunction(weightedInputs[-1]))

        for layer in range(1, len(self.nodeNumbers)-1):
            outputsIncludingBias = np.concatenate([[1], self.outputs[-1]])
            print('layer', layer)
            print('self.weightMatrices[layer]', self.weightMatrices[layer])
            print('outputsIncludingBias', outputsIncludingBias)
            weightedInputs.append(self.weightMatrices[layer] @ outputsIncludingBias)
            self.outputs.append(self.activationFunction(weightedInputs[-1]))
        self.weightedInputs = weightedInputs
        #print('self.outputs', self.outputs[-1])
            
    def calculateErrors(self):
        self.error2 = 0
        if not isinstance(self.targets, list):
                self.error2 += (self.targets - self.outputs[-1])**2
                #print('WRONG')
        else:
            for outputNodeNumber in range(len(self.outputs)):
            #print('outputNodeNumber', outputNodeNumber)
                self.error2 += (self.targets[outputNodeNumber] - \
                           self.outputs[-1][outputNodeNumber])**2
        
    def backward(self):
        # Output deltas
        self.deltaVectors = []
        '''
        deltaValuesOld = np.array([(self.outputs[-1][outputNumber] - self.targets[outputNumber])\
                      *self.differentiateActivationFunction(self.weightedInputs[-1][outputNumber]) \
                      for outputNumber in range(len(self.targets))])
        '''
        print('weightedInputs',self.weightedInputs)
        deltaValues = (self.outputs[-1] - self.targets)* self.differentiateActivationFunction(self.weightedInputs[-1])
        self.deltaVectors.append(deltaValues)
        
        # Hidden deltas
        if not isinstance(self.targets, np.ndarray):
            targetForLoop = np.array([self.targets])
        for layerNumber in range(len(self.nodeNumbers)-2):
            
            deltaVecs = []
            #for j in range(self.nodeNumbers[(layerNumber+1)+1]):
            print('self.weightedInputs[-1]',self.weightedInputs[-1])
            for j in range(2):
                print('j', j)
                deltaSum = 0
                #print('self.targetLength',self.targetLength)
                for k in range(len(self.targetLength)):
                    deltaSum += self.deltaVectors[-1-layerNumber][k] * self.weightMatrices[-1][k,j+1] 
                deltaVecs.append(deltaSum*self.differentiateActivationFunction(self.weightedInputs[-1][j]))
        self.deltaVectors.insert(0, deltaVecs)
        
        print('deltaVectors', self.deltaVectors)
        
        # Output weights
        outputsIncludingBias = np.concatenate([[1], self.outputs[-2]])
        for j in range(self.nodeNumbers[-2] + 1):
            #print(self.targetLength)
            for k in range(len(self.targetLength)):
                self.weightMatrices[-1][k, j] -= \
                self.learningRate*self.deltaVectors[1][k]*outputsIncludingBias[j]
        print('self.weightMatrices',self.weightMatrices)
        # Hidden weights
        print('self.weightMatrices[0]', self.weightMatrices[0])
        print('self.deltaVectors[0]', self.deltaVectors[0])
        print('self.inputs', self.inputs)
        for inputNumber in range(self.nodeNumbers[0]+1):
            for hiddenNodeNumber in range(self.nodeNumbers[1]): # fix indexes
                print('inputNumber', inputNumber, 'hiddenNodeNumber', hiddenNodeNumber)
                
                self.weightMatrices[0][hiddenNodeNumber, inputNumber] -= self.learningRate \
                * self.deltaVectors[0][hiddenNodeNumber]*self.inputs[inputNumber]
     
#def activationFunction(x):
 #   return x
 

def activationFunction(x):
    return 1./(1+np.exp(-x))
 

numberOfNodes = [2,2,1]
activationFunction = activationFunction
learningRate = 0.01
targetMatrix = np.array(((0), (1), (1), (0))).T
inputMatrix = np.array(((-1,0, 0), (-1, 0, 1), (-1,1, 0), (-1,1,1))).T
errorTolerance = 1e-3
maxIterations= 3
test = False
singleOutput = True

nn=NN(numberOfNodes, activationFunction, learningRate, targetMatrix, inputMatrix, errorTolerance, maxIterations, test)
nn.runMultipleInput()
'''
print('nn.weightMatrices',nn.weightMatrices)

nn.forward()
print('nn.weightedInputs',nn.weightedInputs)
print('nn.outputs',nn.outputs)

nn.calculateErrors()
print('nn.error2',nn.error2)

nn.backward()
print('nn.deltaVectors',nn.deltaVectors)
print('weightMatrices', nn.weightMatrices)

nn.run()
'''

self.weightMatrices [array([[-0.46099131, -0.24030389,  0.0155826 ],
       [ 0.01491849,  0.47713365,  0.10239317]]), array([[ 0.29723732, -0.3099767 , -0.14569952]])]
self.inputs [-1  0  0]
self.targets 0
layer 1
self.weightMatrices[layer] [[ 0.29723732 -0.3099767  -0.14569952]]
outputsIncludingBias [1.         0.61324932 0.49627045]
self.error2 [0.25878445]
weightedInputs [array([ 0.46099131, -0.01491849]), array([0.03483795])]
self.weightedInputs[-1] [0.03483795]
j 0
j 1


IndexError: index 1 is out of bounds for axis 0 with size 1

In [90]:
## Slides. Must change derivative inside class to 1
def activationFunction(x):
    return x

numberOfNodes,  activationFunction, learningRate, targetMatrix, inputMatrix, errorTolerance, maxIterations = \
[2,2,2], activationFunction, 0.1, np.array([1, 0]), np.array((1, 0, 1)), 1e-6, 100
test = True


nn2=NN(numberOfNodes, activationFunction, learningRate, targetMatrix, inputMatrix, errorTolerance, maxIterations, test=False)


print('nn2.inputs',nn2.inputs)

print('nn.weightMatrices',nn2.weightMatrices)

nn2.forward()

print('nn.weightedInputs',nn2.weightedInputs)
print('nn.outputs',nn2.outputs)

nn2.calculateErrors()
print('nn.error2',nn2.error2)

nn2.backward()
print('nn.deltaVectors',nn2.deltaVectors)
print('weightMatrices', nn2.weightMatrices)

nn2.run()

self.weightMatrices [array([[-0.04321274,  0.0708538 ,  0.28329251],
       [-0.46904401,  0.2077519 , -0.32281236]]), array([[-0.12063922,  0.06255361, -0.35198749],
       [-0.37555948, -0.18998248,  0.38851104]])]
nn2.inputs [1, 0, 1]
nn.weightMatrices [array([[-0.04321274,  0.0708538 ,  0.28329251],
       [-0.46904401,  0.2077519 , -0.32281236]]), array([[-0.12063922,  0.06255361, -0.35198749],
       [-0.37555948, -0.18998248,  0.38851104]])]
layer 1
self.weightMatrices[layer] [[-0.12063922  0.06255361 -0.35198749]
 [-0.37555948 -0.18998248  0.38851104]]
outputsIncludingBias [ 1.          0.24007977 -0.79185637]
nn.weightedInputs [array([ 0.24007977, -0.79185637]), array([ 0.17310218, -0.72881538])]
nn.outputs [array([ 0.24007977, -0.79185637]), array([ 0.17310218, -0.72881538])]
nn.error2 1.21493186855847
self.weightMatrices[0] [[-0.04321274  0.0708538   0.28329251]
 [-0.46904401  0.2077519  -0.32281236]]
self.deltaVectors[0] [-0.02603167455412636, -0.5020170952018476]
self.inpu

/home/k/.local/lib/python3.6/site-packages/ipykernel_launcher.py:146: RuntimeWarning: overflow encountered in multiply
/home/k/.local/lib/python3.6/site-packages/ipykernel_launcher.py:177: RuntimeWarning: invalid value encountered in double_scalars


In [64]:
def activationFunction(x):
    return 1./(1+np.exp(-x)) 

numberOfNodes,  activationFunction, learningRate, targets, inputs, errorTolerance, maxIterations = \
[2,2,2], activationFunction, 0.1, np.array([1, 0]), np.array((1, 0, 1)), 1e-6, 100000
inputs = np.array((1, 0, 1))

nn3=NN(numberOfNodes, activationFunction, learningRate, targets, inputs, errorTolerance, maxIterations, test=False)
nn3.run()

Iterations:  100000 |Error|:  5.3734953521503506e-05 Output:  [0.99481832 0.00518509]


# In the lecture slides the backpropagation deltas are first presented as $\delta_k = (y_k − t_k )y_k (1 − y_k )$. What does this tell us about the activation function in use?

The Sigmoid function is used as activation function. We have $\delta_k = (y_k -t_k) f'(a)$, where $a$ is the input to the acitivation functino in the output layer. When the Sigmoid is used we have $f'(a_k) = sigmoid(a_k) (1 - sigmoid(a_k)) = y_k(1-y_k)$, which is what we have.

# You are to design an MLP that would learn to hyphenate words correctly. You would have a dictionary that shows correct hyphenation examples for lots of words. Think about the following: 

## What should the input to the neural net be?
Traning face: The full words without hypenation as input and the correct words from the dictionary.
Else: Full words without hypenation.

<mark> Sol:Input = 2 letters.?? So not words?<mark>

## How should this input be encoded to work well with the classifier?
NN needs numbers. One interger per letter.

## How is should the output be encoded?
True and False --> 1d output. True = hyphenate.

## How many layers do you need?
Don' know. Suspect it is not a linear separable problem, so 1 hidden layer as a minimum.

## How many neurons should there be in each layer?
Experiment. Sol: Start with 2N/3, N = inputnumber, in 1st hidden, and N/3 sedond. <mark> Where does this come from?